In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from math import inf
import pandas as pd

In [ ]:
def get_url(type, location, transaction, page):
    return f"https://www.fincaraiz.com.co/{type}/{transaction}/{location}?pagina={page}"

In [ ]:
def get_driver(disable_gui = True):
    options = Options()
    if disable_gui:
        options.add_argument('--headless')
        options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    return(webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options))

In [ ]:
transaction = "venta"
city = "medellin"
department = "antioquia"
location = f"{city}/{department}"
type = "apartaestudios-apartamentos"
houses = []

In [ ]:
def get_information(url: str):
    page_driver = get_driver()
    page_driver.get(url)
    attributes_container = page_driver\
        .find_element(By.ID, "general")\
        .find_elements(By.XPATH, '*')[0]\
        .find_elements(By.XPATH, '*')[1]\
        .find_elements(By.XPATH, '*')[0]
    attributes = attributes_container.find_elements(By.CSS_SELECTOR, "p.jss65.jss74")
    house = {}
    for index in range(len(attributes)):
        if index % 2 != 0:
            continue
        key = attributes[index].text
        value = attributes[index + 1].text
        try:
            if "m²" in value or "&nbsp;" in value or "*":
                value = value.replace("m²", "").replace("&nbsp;", "").replace("*", "").strip()
            house[key] = value
        except Exception as e:
            print(f"Error al obtener el atributo '{key}'")
            print(e)
            pass
    try:
        price = page_driver.find_element(By.TAG_NAME, "aside").find_element(By.XPATH, "//p[text()='Precio total (COP)']/following-sibling::p")
        house["Precio total (COP)"] = price.get_attribute("innerHTML").replace("&nbsp;", "").strip()
    except Exception:
        pass
    houses.append(house)
    page_driver.close()

In [ ]:
main_driver = get_driver()
max_per_page = inf
try:
    for page in range(1, 30):
        main_url = get_url(type, location, transaction, page)
        main_driver.get(main_url)
        articles = main_driver.find_elements(By.TAG_NAME, "article")
        total_articles = len(articles)
        print(f"Página {page}")
        print(f"Esta página contiene {total_articles} elementos")
        houses_iterated = 0
        for index, article in enumerate(articles):
            if houses_iterated == max_per_page:
                break
            print(f"Obteniendo información del inmueble {index + 1} de {total_articles}")
            try:
                url = article.find_element(By.TAG_NAME, "a").get_property("href")
                print(url)
            except Exception:
                print("Error al obtener la URL")
                continue
            get_information(url)
            houses_iterated += 1
finally:
    main_driver.close()


In [ ]:
len(houses)

In [ ]:
houses

In [ ]:
df = pd.DataFrame.from_dict(houses)
df.to_csv(f"house_prices_colombia_{city}.csv")

In [ ]:
df